In [2]:
import pygame
import random
import math
import os
import json

pygame.init()

# ---------- 画面 ----------
W, H = 640, 480
screen = pygame.display.set_mode((W, H))
pygame.display.set_caption("ステージ制シューター（日本語・ショップ/コンボ）")
clock = pygame.time.Clock()

# ---------- フォント ----------
def get_jp_font(size=20, bold=False):
    candidates = [
        "meiryo", "yu gothic ui", "yu gothic", "ms gothic",
        "noto sans cjk jp", "noto sans jp", "hiragino sans", "ipa gothic"
    ]
    path = pygame.font.match_font(candidates)
    f = pygame.font.Font(path, size) if path else pygame.font.SysFont(None, size)
    f.set_bold(bold)
    return f

font = get_jp_font(20)
font_small = get_jp_font(16)
font_tiny = get_jp_font(14)
big_font = get_jp_font(44, bold=True)

# ---------- セーブ ----------
HISCORE_FILE = "hiscore.dat"
UPGRADES_FILE = "upgrades.dat"

def load_hiscore():
    try:
        with open(HISCORE_FILE, "r", encoding="utf-8") as f:
            return int(f.read().strip())
    except:
        return 0

def save_hiscore(sc):
    try:
        with open(HISCORE_FILE, "w", encoding="utf-8") as f:
            f.write(str(sc))
    except:
        pass

def load_upgrades():
    defaults = {
        # 弾アップグレード（永続）
        "multi":0, "rate":0, "damage":0, "pierce":0, "spread":0, "bspeed":0,
        # プレイヤー永続
        "base_speed":5.0, "max_hp":3
    }
    if not os.path.exists(UPGRADES_FILE):
        return defaults.copy()
    try:
        with open(UPGRADES_FILE, "r", encoding="utf-8") as f:
            data = json.load(f)
        for k,v in defaults.items():
            data[k] = data.get(k, v)
        return data
    except:
        return defaults.copy()

def save_upgrades(cfg):
    try:
        with open(UPGRADES_FILE, "w", encoding="utf-8") as f:
            json.dump(cfg, f, ensure_ascii=False)
    except:
        pass

hiscore = load_hiscore()
upg = load_upgrades()

# ---------- プレイヤー ----------
player_size = 28
player_color = (0, 200, 255)
px, py = W//2 - player_size//2, H - player_size - 12

BASE_SPEED_DEFAULT = 5.0
base_speed = max(3.0, min(float(upg["base_speed"]), 8.0))
pspeed = base_speed

max_hp_default = int(upg["max_hp"])
max_hp = max(1, min(max_hp_default, 8))
hp = max_hp

INVINCIBLE_MS = 800
invincible_until = 0

# ボム
bombs = 1
BOMB_DAMAGE = 999

# ---------- 弾 ----------
bullet_color = (255, 255, 0)
bullet_size = 8
BULLET_SPEED_BASE = 9.0

# 上限
LIMITS = {
    "multi":6,     # 発射数段階（発射数=1,2,3,4,5,6,8）
    "rate":7,      # 連射段階（150→…→70ms、下限45ms）
    "damage":4,    # 威力段階（1〜5）
    "pierce":3,    # 貫通段階（1〜4）
    "spread":3,    # 拡散段階（18/26/34/42°）
    "bspeed":3     # 弾速段階
}

# 永続レベル
lvl_multi  = int(upg["multi"])
lvl_rate   = int(upg["rate"])
lvl_damage = int(upg["damage"])
lvl_pierce = int(upg["pierce"])
lvl_spread = int(upg["spread"])
lvl_bspeed = int(upg["bspeed"])

def clamp_levels():
    global lvl_multi,lvl_rate,lvl_damage,lvl_pierce,lvl_spread,lvl_bspeed,base_speed,max_hp
    lvl_multi  = max(0, min(lvl_multi,  LIMITS["multi"]))
    lvl_rate   = max(0, min(lvl_rate,   LIMITS["rate"]))
    lvl_damage = max(0, min(lvl_damage, LIMITS["damage"]))
    lvl_pierce = max(0, min(lvl_pierce, LIMITS["pierce"]))
    lvl_spread = max(0, min(lvl_spread, LIMITS["spread"]))
    lvl_bspeed = max(0, min(lvl_bspeed, LIMITS["bspeed"]))
    base_speed = max(3.0, min(base_speed, 8.0))
    max_hp     = max(1, min(max_hp, 8))

def current_shot_interval_ms():
    table = [150,135,122,110,100,90,80,70]
    ms = table[min(lvl_rate, len(table)-1)]
    return max(45, ms)

def current_bullet_speed():
    mult_table = [1.00, 1.10, 1.22, 1.35]
    return BULLET_SPEED_BASE * mult_table[min(lvl_bspeed, len(mult_table)-1)]

def current_bullet_damage():
    return 1 + lvl_damage

def current_pierce_hits():
    return 1 + lvl_pierce

def current_shot_count_and_angles():
    count_table = [1,2,3,4,5,6,8]
    n = count_table[min(lvl_multi, len(count_table)-1)]
    spread_table = [18, 26, 34, 42]
    spread_deg = spread_table[min(lvl_spread, len(spread_table)-1)]
    if n == 1:
        angs = [math.radians(-90)]
    else:
        start = -90 - spread_deg/2
        step = spread_deg / (n - 1)
        angs = [math.radians(start + i*step) for i in range(n)]
    return n, angs

bullets = []  # [x,y,vx,vy,damage,hits_left]
last_shot_time = -9999

# ---------- 敵 ----------
ENEMY_TYPES = [
    {"name":"small","size":20,"hp":1,"score":1,"coins":1,"spd":(2.8,4.2),"color":(255,120,120)},
    {"name":"mid",  "size":32,"hp":3,"score":3,"coins":3,"spd":(2.0,3.2),"color":(255, 80, 80)},
    {"name":"big",  "size":48,"hp":6,"score":7,"coins":6,"spd":(1.5,2.3),"color":(255, 50, 50)},
]
ENEMY_WEIGHTS = [0.6, 0.28, 0.12]
enemies = []  # dict: x,y,size,hp,score,vy,color,mode,vx,phase,coins

BASE_SPAWN_INTERVAL = 640
SPAWN_INTERVAL_MIN = 240
last_spawn_time = 0
difficulty_start_ticks = pygame.time.get_ticks()

# ステージ制
stage = 1
stage_clear = False
shop_open = False
coins_stage = 0  # ← このステージで稼いだコイン。ショップでのみ使用。終了後は0にリセット。
stage_speed_bonus = 0.0  # ステージごとの常時速度倍率追加

def current_difficulty():
    """経過時間で徐々に難化＋ステージでベースも強化"""
    elapsed = (pygame.time.get_ticks() - difficulty_start_ticks) / 1000.0
    base_mul = 1.0 + stage_speed_bonus  # ステージで加速
    time_mul = min(1.0 + elapsed * 0.008, 1.9)
    speed_mul = base_mul * time_mul
    spawn = max(int(BASE_SPAWN_INTERVAL * (0.986 ** elapsed) / max(1.0, 1.0 + (stage-1)*0.08)), SPAWN_INTERVAL_MIN)
    return spawn, speed_mul

def spawn_enemy():
    etype = random.choices(ENEMY_TYPES, weights=ENEMY_WEIGHTS, k=1)[0]
    size = etype["size"]
    ex = random.randint(0, W - size)
    ey = -size
    _, speed_mul = current_difficulty()
    vmin, vmax = etype["spd"]
    vy = random.uniform(vmin, vmax) * speed_mul
    mode = random.choices(["down","left_right","sin"], weights=[0.5,0.25,0.25])[0]
    vx = random.choice([-1,1]) * random.uniform(0.7,1.5) if mode=="left_right" else 0.0
    phase = random.uniform(0, math.tau)
    enemies.append({
        "x":ex,"y":ey,"size":size,"hp":etype["hp"],"score":etype["score"],
        "vy":vy,"color":etype["color"],"mode":mode,"vx":vx,"phase":phase,
        "coins":etype["coins"]
    })

# ---------- アイテム（永続） ----------
# types: multi, rate, damage, pierce, spread, bspeed, speed_perm, heal, maxhp, bomb, magnet
items = []  # dict: x,y,r,type,vy
ITEM_DROP_RATE = 0.28
ITEM_WEIGHTS = {
    "multi":0.16, "rate":0.16, "damage":0.14, "pierce":0.11,
    "spread":0.11, "bspeed":0.08, "speed_perm":0.07,
    "heal":0.07, "maxhp":0.04, "bomb":0.04, "magnet":0.02
}
MAGNET_MS = 7000
magnet_until = 0

def choose_item_type():
    t = list(ITEM_WEIGHTS.keys())
    w = list(ITEM_WEIGHTS.values())
    return random.choices(t, weights=w, k=1)[0]

def drop_item(x, y, force_type=None):
    if force_type or random.random() < ITEM_DROP_RATE:
        itype = force_type if force_type else choose_item_type()
        items.append({"x":x,"y":y,"r":9,"type":itype,"vy":2.2})

def apply_item_effect(itype):
    global lvl_multi,lvl_rate,lvl_damage,lvl_pierce,lvl_spread,lvl_bspeed
    global base_speed, hp, max_hp, bombs, magnet_until, upg
    if itype == "multi":
        lvl_multi += 1
    elif itype == "rate":
        lvl_rate += 1
    elif itype == "damage":
        lvl_damage += 1
    elif itype == "pierce":
        lvl_pierce += 1
    elif itype == "spread":
        lvl_spread += 1
    elif itype == "bspeed":
        lvl_bspeed += 1
    elif itype == "speed_perm":
        base_speed *= 1.08
    elif itype == "heal":
        hp = min(max_hp, hp + 1)
    elif itype == "maxhp":
        max_hp += 1
        hp = min(max_hp, hp + 1)
    elif itype == "bomb":
        bombs = min(bombs + 1, 5)
    elif itype == "magnet":
        magnet_until = pygame.time.get_ticks() + MAGNET_MS

    clamp_levels()
    upg.update({
        "multi":lvl_multi, "rate":lvl_rate, "damage":lvl_damage, "pierce":lvl_pierce,
        "spread":lvl_spread, "bspeed":lvl_bspeed,
        "base_speed":base_speed, "max_hp":max_hp
    })
    save_upgrades(upg)

# ---------- 爆発 ----------
explosions = []  # dict: x,y,r,max_r,life,life_max,color
def add_explosion(cx, cy, size):
    max_r = int(size * 1.25)
    explosions.append({"x":cx,"y":cy,"r":6,"max_r":max_r,"life":0,"life_max":18,"color":(255,200,120)})

# ---------- ボス（ステージゴール） ----------
boss = None
boss_bullets = []  # [x,y,vx,vy,r,color]
BOSS_SCORE_STEP = 30              # 参考値（未使用でもよい）
BOSS_BASE_HP = 100
def spawn_boss():
    base_hp = int(BOSS_BASE_HP * (1.0 + 0.25*(stage-1)))  # ステージでHP増加
    return {"x":W//2-60,"y":-120,"w":120,"h":80,"hp":base_hp,"hp_max":base_hp,"phase":0,"vy":1.5,"timer":0,"cool":0}

def boss_shoot_pattern(b):
    p = (b["phase"] // 300) % 3
    if p == 0:
        n = 18
        for i in range(n):
            ang = (math.tau * i) / n
            spd = 2.6
            boss_bullets.append([b["x"]+b["w"]/2, b["y"]+b["h"]/2, math.cos(ang)*spd, math.sin(ang)*spd, 5, (255,170,80)])
    elif p == 1:
        for i in range(-4, 5):
            ang = math.radians(90 + i*10)
            spd = 3.0
            boss_bullets.append([b["x"]+b["w"]/2, b["y"]+b["h"], math.cos(ang)*spd, math.sin(ang)*spd, 5, (180,255,120)])
    else:
        for _ in range(6):
            dx = (px + player_size/2) - (b["x"]+b["w"]/2)
            dy = (py + player_size/2) - (b["y"]+b["h"]/2)
            ang = math.atan2(dy, dx)
            spd = 3.4
            boss_bullets.append([b["x"]+b["w"]/2, b["y"]+b["h"]/2, math.cos(ang)*spd, math.sin(ang)*spd, 5, (255,120,200)])

# ---------- コンボ（キル継続で上昇、切れたら即リセット） ----------
COMBO_STEP = 0.2
COMBO_MAX = 6.0
COMBO_RESET_MS = 2500  # この時間キルが無いと即 x1.0 に戻る
combo_mult = 1.0
last_kill_time = pygame.time.get_ticks()

def on_kill_award(base_score, base_coins):
    """キル時のスコア＆コイン（ステージコイン）加算。コンボ上昇。"""
    global score, coins_stage, combo_mult, last_kill_time
    gained = max(1, int(round(base_score * combo_mult)))
    score += gained
    coins_stage += base_coins
    combo_mult = min(COMBO_MAX, round(combo_mult + COMBO_STEP, 2))
    last_kill_time = pygame.time.get_ticks()
    return gained

def tick_combo_reset(now):
    global combo_mult
    if now - last_kill_time > COMBO_RESET_MS:
        combo_mult = 1.0

# ---------- スコア/状態 ----------
score = 0
game_over = False
paused = False
title = True

# ---------- 便利関数 ----------
def rect_collide(ax, ay, aw, ah, bx, by, bw, bh):
    return (ax < bx + bw and bx < ax + aw and ay < by + bh and by < ay + ah)

def make_player_shots(cx, cy):
    n, angs = current_shot_count_and_angles()
    spd = current_bullet_speed()
    dmg = current_bullet_damage()
    pier = current_pierce_hits()
    shots = []
    for ang in angs:
        vx = math.cos(ang) * spd
        vy = math.sin(ang) * spd
        shots.append([cx - bullet_size//2, cy, vx, vy, dmg, pier])
    return shots

def reset_run_for_stage(new_stage):
    """ステージ開始時の初期化（永続強化は保持）。"""
    global px, py, pspeed, hp, max_hp, invincible_until, bombs
    global bullets, enemies, items, explosions
    global boss, boss_bullets, stage, stage_clear, coins_stage
    global last_shot_time, last_spawn_time, difficulty_start_ticks, stage_speed_bonus

    stage = new_stage
    stage_clear = False
    coins_stage = 0

    px, py = W//2 - player_size//2, H - player_size - 12
    invincible_until = 0
    bombs = 1

    pspeed = base_speed
    hp = max_hp

    bullets = []
    enemies = []
    items = []
    explosions = []

    boss = None
    boss_bullets = []

    last_shot_time = -9999
    last_spawn_time = 0
    difficulty_start_ticks = pygame.time.get_ticks()

    # ステージが上がると基礎スピード倍率も上がる
    stage_speed_bonus = 0.12 * (stage - 1)

def apply_bomb():
    global enemies, boss_bullets, boss
    boss_bullets.clear()
    dead_idx = set()
    for i, e in enumerate(enemies):
        e["hp"] -= BOMB_DAMAGE
        if e["hp"] <= 0:
            dead_idx.add(i)
            on_kill_award(e["score"], e["coins"])
            add_explosion(e["x"] + e["size"]/2, e["y"] + e["size"]/2, e["size"])
            drop_item(e["x"] + e["size"]/2, e["y"] + e["size"]/2)
    enemies = [e for i, e in enumerate(enemies) if i not in dead_idx]

    if boss is not None:
        boss["hp"] -= max(20, boss["hp_max"] // 5)
        if boss["hp"] <= 0:
            add_explosion(boss["x"] + boss["w"]/2, boss["y"] + boss["h"]/2, max(boss["w"], boss["h"]))
            on_kill_award(25, 20)
            drop_item(boss["x"] + boss["w"]/2, boss["y"] + boss["h"]/2, force_type="multi")
            boss = None

# ---------- ショップ（ステージ間だけ） ----------
# 価格関数
def price_linear(base, step, cur, cap=None):
    if cap is not None and cur >= cap:
        return None
    return base + step * cur

def price_bulk5(base, step, cur, cap):
    """+5購入の合計価格（capを超える分は買えない）"""
    if cur >= cap:
        return None, 0
    can = min(5, cap - cur)
    total = 0
    for i in range(can):
        total += base + step * (cur + i)
    return total, can

# 購入アクション
def buy_multi(n=1):
    global lvl_multi
    lvl_multi = min(lvl_multi + n, LIMITS["multi"])

def buy_rate(n=1):
    global lvl_rate
    lvl_rate = min(lvl_rate + n, LIMITS["rate"])

def buy_damage(n=1):
    global lvl_damage
    lvl_damage = min(lvl_damage + n, LIMITS["damage"])

def buy_pierce(n=1):
    global lvl_pierce
    lvl_pierce = min(lvl_pierce + n, LIMITS["pierce"])

def buy_spread(n=1):
    global lvl_spread
    lvl_spread = min(lvl_spread + n, LIMITS["spread"])

def buy_bspeed(n=1):
    global lvl_bspeed
    lvl_bspeed = min(lvl_bspeed + n, LIMITS["bspeed"])

def buy_speed_perm(n=1):
    global base_speed
    for _ in range(n):
        base_speed = min(8.0, base_speed * 1.08)

def buy_maxhp(n=1):
    global max_hp, hp
    for _ in range(n):
        if max_hp < 8:
            max_hp += 1
            hp = min(max_hp, hp + 1)

def buy_bomb(n=1):
    global bombs
    bombs = min(5, bombs + n)

SHOP_ITEMS = [
    # (番号, 名前, 価格(base, step), getter, 上限, 買う関数)
    ("1", "弾数Lv",    (30, 20), lambda: lvl_multi,  LIMITS["multi"], buy_multi),
    ("2", "連射Lv",    (40, 20), lambda: lvl_rate,   LIMITS["rate"],  buy_rate),
    ("3", "威力Lv",    (50, 30), lambda: lvl_damage, LIMITS["damage"], buy_damage),
    ("4", "貫通Lv",    (60, 40), lambda: lvl_pierce, LIMITS["pierce"], buy_pierce),
    ("5", "拡散Lv",    (30, 15), lambda: lvl_spread, LIMITS["spread"], buy_spread),
    ("6", "弾速Lv",    (30, 15), lambda: lvl_bspeed, LIMITS["bspeed"], buy_bspeed),
    ("7", "移動速度",  (70, 25), lambda: int((base_speed-3.0)*100), None,        buy_speed_perm),
    ("8", "最大HP+1",  (100,50), lambda: max_hp-1,   8,               buy_maxhp),
    ("9", "ボム+1",    (60, 10), lambda: bombs,      5,               buy_bomb),
]

def open_shop_after_stage():
    global shop_open, paused
    shop_open = True
    paused = True  # ゲームは停止

def close_shop_and_start_next_stage():
    """購入終了。永続保存して次ステージへ。ステージコインはリセット。"""
    global shop_open, paused, coins_stage, stage
    shop_open = False
    paused = False
    coins_stage = 0
    stage += 1
    # 保存（永続強化）
    upg.update({
        "multi":lvl_multi, "rate":lvl_rate, "damage":lvl_damage, "pierce":lvl_pierce,
        "spread":lvl_spread, "bspeed":lvl_bspeed,
        "base_speed":base_speed, "max_hp":max_hp
    })
    save_upgrades(upg)
    reset_run_for_stage(stage)

def try_buy(slot_key, bulk5=False):
    """bulk5=True で +5Lv 購入（Shift+数字）"""
    global coins_stage
    for key, name, (base, step), getlv, cap, action in SHOP_ITEMS:
        if key != slot_key:
            continue
        cur = getlv()
        if bulk5 and cap is not None:
            total, can = price_bulk5(base, step, cur, cap)
            if total is None or can <= 0:
                return f"{name} は最大です"
            if coins_stage >= total:
                coins_stage -= total
                action(can)
                clamp_levels()
                return f"{name} を +{can} 購入！ (-{total}c)"
            else:
                return "コインが足りません"
        else:
            price = price_linear(base, step, cur, cap)
            if price is None:
                return f"{name} は最大です"
            if coins_stage >= price:
                coins_stage -= price
                action(1)
                clamp_levels()
                return f"{name} を購入！ (-{price}c)"
            else:
                return "コインが足りません"
    return "無効な選択"

# ---------- 初期化 ----------
def boot_title():
    global title
    title = True

def start_from_title():
    # タイトル→ステージ1開始
    reset_run_for_stage(1)

boot_title()

# ---------- メインループ ----------
running = True
while running:
    now = pygame.time.get_ticks()
    spawn_interval, speed_mul = current_difficulty()

    # --- イベント ---
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if title:
                if event.key == pygame.K_RETURN:
                    title = False
                    start_from_title()
                elif event.key == pygame.K_t:
                    # 永続リセット
                    upg = {
                        "multi":0,"rate":0,"damage":0,"pierce":0,"spread":0,"bspeed":0,
                        "base_speed":BASE_SPEED_DEFAULT,"max_hp":3
                    }
                    save_upgrades(upg)
                    lvl_multi=lvl_rate=lvl_damage=lvl_pierce=lvl_spread=lvl_bspeed=0
                    base_speed=BASE_SPEED_DEFAULT; max_hp=3
            else:
                if shop_open:
                    # ショップ内操作
                    if event.key == pygame.K_RETURN:
                        close_shop_and_start_next_stage()
                    elif event.key == pygame.K_ESCAPE:
                        close_shop_and_start_next_stage()
                    else:
                        # 数字キー購入 / Shift+数字で +5
                        bulk5 = (event.mod & pygame.KMOD_SHIFT) != 0
                        if event.unicode in [k for (k, *_rest) in SHOP_ITEMS]:
                            msg = try_buy(event.unicode, bulk5=bulk5)
                            pygame.display.set_caption(f"ショップ: {msg}")
                else:
                    # ゲーム中
                    if event.key == pygame.K_p and not game_over:
                        paused = not paused
                    if event.key == pygame.K_b and not paused and not game_over and bombs > 0:
                        apply_bomb()
                        bombs -= 1

    keys = pygame.key.get_pressed()

    # ---------- タイトル ----------
    if title:
        screen.fill((6, 8, 15))
        screen.blit(big_font.render("ステージ制シューター", True, (255, 240, 200)), (W//2-210, 100))
        screen.blit(font.render("Enter: スタート   P: ポーズ   B: ボム   Esc: 終了", True, (230, 230, 240)), (W//2-220, 168))
        screen.blit(font.render("強化は保存され、次回起動時も引き継がれます。", True, (210, 230, 255)), (W//2-220, 196))
        screen.blit(font.render("T: タイトル画面で強化リセット", True, (210, 230, 255)), (W//2-160, 222))
        screen.blit(font.render(f"ハイスコア: {hiscore}", True, (255,255,255)), (10, 10))
        guide = "操作: 矢印=移動 / スペース=射撃 / B=ボム / P=ポーズ / （ショップはステージ間）"
        screen.blit(font_tiny.render(guide, True, (200,210,220)), (W - 8 - font_tiny.size(guide)[0], H - 6 - font_tiny.get_height()))
        pygame.display.flip()
        clock.tick(60)
        continue

    # ---------- ショップ画面 ----------
    if shop_open:
        # 背景を半透明で覆う
        overlay = pygame.Surface((W, H), pygame.SRCALPHA)
        overlay.fill((0, 0, 0, 180))
        screen.blit(overlay, (0, 0))

        box_w, box_h = 540, 360
        box_x, box_y = (W - box_w)//2, (H - box_h)//2
        pygame.draw.rect(screen, (30,30,45), (box_x, box_y, box_w, box_h), border_radius=12)
        pygame.draw.rect(screen, (90,90,140), (box_x, box_y, box_w, box_h), width=2, border_radius=12)

        title_txt = big_font.render(f"ショップ - ステージ {stage} 完了！", True, (255,255,255))
        screen.blit(title_txt, (box_x + (box_w - title_txt.get_width())//2, box_y + 12))

        screen.blit(font.render(f"使えるコイン: {coins_stage}", True, (255,230,120)), (box_x + 18, box_y + 70))
        screen.blit(font_small.render("数字キー=購入 / Shift+数字=+5Lv購入 / Enter=次のステージ / Esc=スキップ", True, (210,220,230)), (box_x + 18, box_y + 100))

        start_y = box_y + 140
        line_h = 28
        for i,(key,name,(base,step),getlv,cap,action) in enumerate(SHOP_ITEMS):
            y = start_y + i*line_h
            cur = getlv()
            level_str = f"(Lv:{cur})" if cap is not None else ""
            # 通常価格
            p1 = price_linear(base, step, cur, cap)
            if p1 is None:
                txt = f"[{key}] {name} {level_str} … MAX"
                color = (160, 220, 255)
            else:
                # +5価格
                if cap is not None:
                    p5, can = price_bulk5(base, step, cur, cap)
                    p5_txt = f" / +5:{p5}c" if p5 is not None and can==5 else (f" / +{can}:{p5}c" if p5 is not None and can>0 else "")
                else:
                    p5_txt = ""
                txt = f"[{key}] {name} {level_str} … {p1}c{p5_txt}"
                color = (200, 255, 200) if coins_stage >= p1 else (255, 180, 180)
            screen.blit(font_small.render(txt, True, color), (box_x + 24, y))

        pygame.display.flip()
        clock.tick(60)
        continue

    # ---------- ポーズ ----------
    if paused:
        overlay = pygame.Surface((W, H), pygame.SRCALPHA)
        overlay.fill((0, 0, 0, 120))
        screen.blit(overlay, (0, 0))
        txt = big_font.render("一時停止中", True, (255, 255, 255))
        screen.blit(txt, (W//2 - txt.get_width()//2, H//2 - 28))
        guide = "操作: 矢印=移動 / スペース=射撃 / B=ボム / P=ポーズ / Esc=終了"
        screen.blit(font_tiny.render(guide, True, (220,230,240)), (W - 8 - font_tiny.size(guide)[0], H - 6 - font_tiny.get_height()))
        pygame.display.flip()
        clock.tick(60)
        continue

    # ---------- ゲーム進行（ステージ中） ----------
    if not game_over:
        pspeed = base_speed

        # コンボリセット判定
        tick_combo_reset(now)

        # 移動
        if keys[pygame.K_LEFT]:  px -= pspeed
        if keys[pygame.K_RIGHT]: px += pspeed
        if keys[pygame.K_UP]:    py -= pspeed
        if keys[pygame.K_DOWN]:  py += pspeed
        px = max(0, min(W - player_size, px))
        py = max(0, min(H - player_size, py))

        # 射撃
        shot_interval = current_shot_interval_ms()
        if keys[pygame.K_SPACE] and (now - last_shot_time >= shot_interval):
            cx = px + player_size // 2
            cy = py
            bullets += make_player_shots(cx, cy)
            last_shot_time = now

        # 弾更新
        new_bul = []
        for x,y,vx,vy,dm,ph in bullets:
            x += vx; y += vy
            if -20 <= x <= W+20 and -20 <= y <= H+20:
                new_bul.append([x,y,vx,vy,dm,ph])
        bullets = new_bul

        # 敵スポーン（ボス不在時）
        if boss is None and now - last_spawn_time >= spawn_interval:
            spawn_enemy()
            last_spawn_time = now

        # 敵更新
        for e in enemies:
            if e["mode"] == "down":
                e["y"] += e["vy"]
            elif e["mode"] == "left_right":
                e["y"] += e["vy"]; e["x"] += e["vx"]
                if e["x"] <= 0 or e["x"] + e["size"] >= W:
                    e["vx"] *= -1
            else:
                e["y"] += e["vy"]; e["phase"] += 0.06
                e["x"] += math.sin(e["phase"]) * 2.0
        enemies = [e for e in enemies if e["y"] < H + e["size"]]

        # ボス登場条件：一定時間経過 or スコアしきい値でもOKだが、ここは時間で出す
        # 例：ステージ開始から20秒で出現
        if boss is None:
            elapsed = (pygame.time.get_ticks() - difficulty_start_ticks) / 1000.0
            if elapsed > 20.0:
                boss = spawn_boss()
                enemies.clear()

        # 弾 vs 敵（貫通）
        hit_bul_count = {}
        dead_idx = set()
        for ei, e in enumerate(enemies):
            erect = (e["x"], e["y"], e["size"], e["size"])
            for bi, b in enumerate(bullets):
                bx,by,bvx,bvy,dm,ph = b
                if rect_collide(bx,by,bullet_size,bullet_size,*erect):
                    e["hp"] -= dm
                    hit_bul_count[bi] = hit_bul_count.get(bi, 0) + 1
                    if e["hp"] <= 0:
                        dead_idx.add(ei)
                        add_explosion(e["x"]+e["size"]/2, e["y"]+e["size"]/2, e["size"])
                        drop_item(e["x"]+e["size"]/2, e["y"]+e["size"]/2)
                        on_kill_award(e["score"], e["coins"])
        # 弾の寿命（貫通）
        new_bul = []
        for bi, b in enumerate(bullets):
            bx,by,bvx,bvy,dm,ph = b
            hits = hit_bul_count.get(bi, 0)
            ph_after = ph - hits
            if ph_after > 0:
                new_bul.append([bx,by,bvx,bvy,dm,ph_after])
        bullets = new_bul
        enemies = [e for i,e in enumerate(enemies) if i not in dead_idx]

        # アイテム更新・取得
        for it in items:
            if now < magnet_until:
                dx = (px + player_size/2) - it["x"]
                dy = (py + player_size/2) - it["y"]
                dist = math.hypot(dx, dy) + 1e-6
                it["x"] += dx / dist * 2.5
                it["y"] += dy / dist * 2.5
            else:
                it["y"] += it["vy"]
        items = [it for it in items if it["y"] < H + it["r"]]
        for it in items[:]:
            pcx,pcy = px + player_size/2, py + player_size/2
            dx,dy = it["x"]-pcx, it["y"]-pcy
            if dx*dx + dy*dy <= (it["r"] + player_size/2)**2:
                apply_item_effect(it["type"])
                items.remove(it)

        # 爆発更新
        new_exp = []
        for ex in explosions:
            ex["life"] += 1
            t = ex["life"]/ex["life_max"]
            ex["r"] = int(6 + (ex["max_r"] - 6) * t)
            if ex["life"] < ex["life_max"]:
                new_exp.append(ex)
        explosions = new_exp

        # プレイヤー被弾（雑魚）
        if now >= invincible_until:
            prect = (px,py,player_size,player_size)
            for e in enemies:
                if rect_collide(*prect, e["x"], e["y"], e["size"], e["size"]):
                    hp -= 1
                    invincible_until = now + INVINCIBLE_MS
                    combo_mult = 1.0  # 被弾でコンボも切る
                    break

        # ボス処理
        if boss is not None:
            b = boss
            if b["y"] < 40:
                b["y"] += b["vy"]
            else:
                b["phase"] += 1
                b["cool"] -= 1
                b["x"] = W/2 - b["w"]/2 + math.sin(b["phase"]*0.02) * 160
                if b["cool"] <= 0:
                    boss_shoot_pattern(b)
                    b["cool"] = 45

            # 自弾→ボス（貫通）
            bx,by,bw,bh = b["x"],b["y"],b["w"],b["h"]
            new_bul = []
            for x,y,vx,vy,dm,ph in bullets:
                if rect_collide(x,y,bullet_size,bullet_size,bx,by,bw,bh):
                    b["hp"] -= dm
                    ph -= 1
                    if ph > 0:
                        new_bul.append([x,y,vx,vy,dm,ph])
                else:
                    new_bul.append([x,y,vx,vy,dm,ph])
            bullets = new_bul

            # ボス弾更新・被弾
            new_bb = []
            for x,y,vx,vy,r,col in boss_bullets:
                x += vx; y += vy
                if -20 <= x <= W+20 and -20 <= y <= H+20:
                    new_bb.append([x,y,vx,vy,r,col])
            boss_bullets = new_bb

            if now >= invincible_until:
                for x,y,vx,vy,r,col in boss_bullets:
                    if rect_collide(px,py,player_size,player_size,x-r,y-r,r*2,r*2):
                        hp -= 1
                        invincible_until = now + INVINCIBLE_MS
                        combo_mult = 1.0
                        break

            # 撃破
            if b["hp"] <= 0:
                add_explosion(bx+bw/2, by+bh/2, max(bw,bh))
                on_kill_award(25, 20)
                drop_item(bx+bw/2, by+bh/2, force_type="multi")
                boss = None
                boss_bullets.clear()
                # ステージクリア → ショップへ
                stage_clear = True
                open_shop_after_stage()

        # HPゼロ
        if hp <= 0:
            game_over = True
            if score > hiscore:
                hiscore = score
                save_hiscore(hiscore)

    # ---------- 描画 ----------
    screen.fill((8, 10, 16))

    # プレイヤー（無敵点滅）
    if not game_over and not shop_open:
        draw_player = True
        if pygame.time.get_ticks() < invincible_until:
            draw_player = ((pygame.time.get_ticks() // 60) % 2) == 0
        if draw_player:
            pygame.draw.rect(screen, player_color, (px, py, player_size, player_size))

    # 自弾
    for x,y,vx,vy,dm,ph in bullets:
        pygame.draw.rect(screen, bullet_color, (x, y, bullet_size, bullet_size))

    # 敵
    for e in enemies:
        pygame.draw.rect(screen, e["color"], (e["x"], e["y"], e["size"], e["size"]))
        base_hp = next(t["hp"] for t in ENEMY_TYPES if t["size"] == e["size"])
        hp_ratio = max(0.0, min(1.0, e["hp"]/base_hp))
        bar_w, bar_h = e["size"], 3
        bx, by = e["x"], e["y"] - 5
        pygame.draw.rect(screen, (60,60,60), (bx, by, bar_w, bar_h))
        pygame.draw.rect(screen, (0,255,0), (bx, by, int(bar_w*hp_ratio), bar_h))

    # ボス
    if boss is not None:
        b = boss
        pygame.draw.rect(screen, (200,80,200), (b["x"], b["y"], b["w"], b["h"]))
        ratio = max(0.0, b["hp"]/b["hp_max"])
        pygame.draw.rect(screen, (60,60,90), (W//2-160, 8, 320, 10))
        pygame.draw.rect(screen, (255,100,255), (W//2-160, 8, int(320*ratio), 10))
    for x,y,vx,vy,r,col in boss_bullets:
        pygame.draw.circle(screen, col, (int(x), int(y)), r)

    # アイテム
    for it in items:
        t = it["type"]
        if t == "multi":         color = (255,220,140)
        elif t == "rate":        color = (200,230,255)
        elif t == "damage":      color = (255,180,120)
        elif t == "pierce":      color = (255,140,200)
        elif t == "spread":      color = (255,235,160)
        elif t == "bspeed":      color = (200,255,200)
        elif t == "speed_perm":  color = (180,255,200)
        elif t == "heal":        color = (255,120,120)
        elif t == "maxhp":       color = (255,180,180)
        elif t == "bomb":        color = (255,200,60)
        else:                    color = (160,220,255)  # magnet
        pygame.draw.circle(screen, color, (int(it["x"]), int(it["y"])), it["r"])

    # 爆発
    for ex in explosions:
        t = ex["life"]/ex["life_max"]
        alpha = max(0, int(255*(1.0-t)))
        surf = pygame.Surface((ex["max_r"]*2, ex["max_r"]*2), pygame.SRCALPHA)
        pygame.draw.circle(surf, (*ex["color"], alpha), (ex["max_r"], ex["max_r"]), ex["r"])
        screen.blit(surf, (ex["x"]-ex["max_r"], ex["y"]-ex["max_r"]))

    # UI：スコア／ハイスコア／ステージ／ステージコイン
    screen.blit(font.render(f"スコア: {score}   ハイスコア: {hiscore}   ステージ: {stage}   コイン: {coins_stage}", True, (255,255,255)), (10, 8))
    # HP
    for i in range(max_hp):
        x = 10 + i*16; y = 32
        col = (255,80,100) if i < hp else (90,60,70)
        pygame.draw.rect(screen, col, (x, y, 12, 12), border_radius=3)
    # 爆弾
    screen.blit(font_small.render(f"ボム: {bombs}", True, (255,230,120)), (10, 50))

    # 弾ステータス（左下）— nonlocal 使わずに for で描画
    y0 = H - 80
    for text in [
        f"弾数Lv:{lvl_multi}  連射:{current_shot_interval_ms()}ms",
        f"威力Lv:{lvl_damage}  貫通Lv:{lvl_pierce}",
        f"拡散Lv:{lvl_spread}  弾速Lv:{lvl_bspeed}",
        f"移動速度:{base_speed:.1f}",
    ]:
        screen.blit(font_tiny.render(text, True, (200,230,255)), (10, y0))
        y0 += 14

    # コンボ表示（上中央）
    combo_text = f"COMBO x{combo_mult:.1f}"
    c_col = (255, 210, 120) if combo_mult < 2.0 else (255, 120, 120)
    c_img = big_font.render(combo_text, True, c_col)
    screen.blit(c_img, (W//2 - c_img.get_width()//2, 34))

    # 難易度（右上）
    di_txt = font_small.render(f"出現 ~{spawn_interval}ms / 速度 x{speed_mul:.2f}", True, (170,180,200))
    screen.blit(di_txt, (W-280, 8))

    # 右下 操作ガイド
    guide = "操作: 矢印=移動 / Space=射撃 / B=ボム / P=ポーズ / Esc=終了（ショップはステージ間）"
    screen.blit(font_tiny.render(guide, True, (190, 200, 210)), (W - 8 - font_tiny.size(guide)[0], H - 6 - font_tiny.get_height()))

    # ゲームオーバー
    if game_over:
        over1 = big_font.render("ゲームオーバー", True, (255,200,200))
        over2 = font_small.render("R: やり直し（強化は保持）   Esc: 終了", True, (240,220,220))
        screen.blit(over1, (W//2 - over1.get_width()//2, H//2 - 30))
        screen.blit(over2, (W//2 - over2.get_width()//2, H//2 + 18))
        if keys[pygame.K_r]:
            # ステージ1から再開
            reset_run_for_stage(1)
            game_over = False
        if keys[pygame.K_ESCAPE]:
            running = False

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
